In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

100%|██████████| 69120/69120 [00:04<00:00, 16731.40it/s]


In [7]:
COLUMNS = [
    # "facts",
    "optimiser",
    "features",
    "iterations",
    "feature_pruning",
    "data_pruning",
]

def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "all"]
    df = df[df.features == "wl"]
    df = df[df.data_pruning == "none"]
    # df = df[df.feature_pruning.isin({"none", "collapse-all"})]
    return df

for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    # df = df.fillna(0)
    # replace 0 with NaN
    df = df.replace(0, -1)
    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)

solved


domain                                                       blocksworld  \
optimiser features iterations feature_pruning  data_pruning                
rank-svm  wl       2          collapse-all     none                   24   
                              collapse-layer   none                   22   
                              collapse-layer-x none                   16   
                              none             none                   23   
                   4          collapse-all     none                   22   
                              collapse-layer   none                   22   
                              collapse-layer-x none                   17   
                              none             none                   21   
                   6          collapse-all     none                   22   
                              collapse-layer   none                   23   
                              collapse-layer-x none                   16   
                              none             none                   -1   
svr       wl       2          collapse-all     none                    9   
                              collapse-layer   none                   11   
                              collapse-layer-x none                   16   
                              none             none                   10   
                   4          collapse-all     none                   20   
                              collapse-layer   none                   12   
                              collapse-layer-x none                   16   
                              none             none                   20   
                   6          collapse-all     none                   21   
                              collapse-layer   none                   10   
                              collapse-layer-x none                   15   
                              none             none                   21   

domain                                                       childsnack  \
optimiser features iterations feature_pruning  data_pruning               
rank-svm  wl       2          collapse-all     none                  18   
                              collapse-layer   none                  10   
                              collapse-layer-x none                  18   
                              none             none                   9   
                   4          collapse-all     none                  13   
                              collapse-layer   none                  11   
                              collapse-layer-x none                  17   
                              none             none                   9   
                   6          collapse-all     none                  12   
                              collapse-layer   none                  11   
                              collapse-layer-x none                  19   
                              none             none                   6   
svr       wl       2          collapse-all     none                   2   
                              collapse-layer   none                   2   
                              collapse-layer-x none                   2   
                              none             none                   2   
                   4          collapse-all     none                   2   
                              collapse-layer   none                   2   
                              collapse-layer-x none                   2   
                              none             none                   2   
                   6          collapse-all     none                   2   
                              collapse-layer   none                   2   
                              collapse-layer-x none                   2   
                              none             none                   2   

domain                                                       ferry  mico

In [4]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"]
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="feature_pruning",
            line_dash="feature_pruning",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)